<a href="https://colab.research.google.com/github/franz6ko/natural-lenguage-processing/blob/master/challenge-1-word2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://www.inove.com.ar"><img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center"></a>


# Procesamiento de lenguaje natural
## Word2vect


In [1]:
import numpy as np

In [2]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * (np.linalg.norm(b)))

### Datos

In [3]:
corpus = np.array(['que dia es hoy', 'martes el dia de hoy es martes', 'martes muchas gracias'])

### 1 - Obtener el vocabulario del corpus (los términos utilizados)
- Cada documento transformarlo en una lista de términos
- Armar un vector de términos no repetidos de todos los documentos

In [12]:
corpus_list = [ doc.split(' ') for doc in corpus ]
print("Docs como listas de terminos:")
print(corpus_list)

vocab = set()
for doc in corpus:
  vocab.update(doc.split(' '))
print("Vocabulario:")
print(vocab)

Docs como listas de terminos:
[['que', 'dia', 'es', 'hoy'], ['martes', 'el', 'dia', 'de', 'hoy', 'es', 'martes'], ['martes', 'muchas', 'gracias']]
Vocabulario:
{'martes', 'el', 'gracias', 'que', 'hoy', 'dia', 'es', 'muchas', 'de'}


### 2- OneHot encoding
Dada una lista de textos, devolver una matriz con la representación oneHotEncoding de estos

In [16]:
ohe = np.zeros((len(corpus_list), len(vocab)))

for i, doc in enumerate(corpus_list):
  for j, term in enumerate(vocab):
    ohe[i,j] = term in doc

print(ohe)

[[0. 0. 0. 1. 1. 1. 1. 0. 0.]
 [1. 1. 0. 0. 1. 1. 1. 0. 1.]
 [1. 0. 1. 0. 0. 0. 0. 1. 0.]]


### 3- Vectores de frecuencia
Dada una lista de textos, devolver una matriz con la representación de frecuencia de estos

In [18]:
tf = np.zeros((len(corpus_list), len(vocab)))

for i, doc in enumerate(corpus_list):
  for j, term in enumerate(vocab):
    tf[i,j] = doc.count(term)

print(tf)

[[0. 0. 0. 1. 1. 1. 1. 0. 0.]
 [2. 1. 0. 0. 1. 1. 1. 0. 1.]
 [1. 0. 1. 0. 0. 0. 0. 1. 0.]]


### 4- TF-IDF
Dada una lista de textos, devolver una matriz con la representacion TFIDF

In [28]:
df = ohe.sum(axis=0)
n = len(corpus_list)
idf = np.log10(n/df)
tf_idf = (tf * idf).round(decimals=2)
print(tf_idf)

[[0.   0.   0.   0.48 0.18 0.18 0.18 0.   0.  ]
 [0.35 0.48 0.   0.   0.18 0.18 0.18 0.   0.48]
 [0.18 0.   0.48 0.   0.   0.   0.   0.48 0.  ]]


### 5 - Comparación de documentos
Realizar una funcion que reciba el corpus y el índice de un documento y devuelva los documentos ordenados por la similitud coseno

In [104]:
comp = np.zeros((n, n))

for i in range(n):
  for j in range(n):
    comp[i,j] = cosine_similarity(tf_idf[i], tf_idf[j])

print(comp)

[[1.         0.20586393 0.        ]
 [0.20586393 1.         0.10874642]
 [0.         0.10874642 1.        ]]


In [113]:
comp_up = np.triu(comp + 1e-15, k=1)
print(comp_up.round(decimals=4))

[[0.     0.2059 0.    ]
 [0.     0.     0.1087]
 [0.     0.     0.    ]]


In [114]:
lim = int(n*(n-1)/2)
order = np.array(np.unravel_index(np.argsort(comp_up.flatten() * -1), comp_up.shape))[:, :lim]
print(order)

[[0 1 0]
 [1 2 2]]


In [115]:
for i in range(lim):
  print("Cosine similarity: " + str(comp[order[0, i]][order[1, i]].round(decimals=4)) + " | " + corpus[order[0, i]] + " <-> " + corpus[order[1, i]])

Cosine similarity: 0.2059 | que dia es hoy <-> martes el dia de hoy es martes
Cosine similarity: 0.1087 | martes el dia de hoy es martes <-> martes muchas gracias
Cosine similarity: 0.0 | que dia es hoy <-> martes muchas gracias
